In [ ]:
#You need the Majorana kernel to run this code, the kernel can be setup using the shift image
import numpy as np
from root_numpy import array, root2array, tree2array
import matplotlib
matplotlib.use('Agg')
import matplotlib.pyplot as plt
from tqdm import tqdm
import pickle
import os

from ROOT import GATDataSet, TFile, TTree, MJTMSWaveform, MGTWaveform
from settings import *

In [ ]:
#Convert ROOT vector to numpy array, unfortunately this is slow, but np.array(ROOTTArray) does not work for unknown reason.
def wfconverter(wf):
    output = []
    for i in range(wf.size()):
        output.append(wf[i])
    return np.array(output)

#Select DEP/SEP events using skim file, then extract waveform using Clint's code
def get_peak(filename, elow, ehi,skimPath,subsample=0.1):
    '''
    filename: the output .pickle file name you'd like to save
    elow: the lower energy bound of the peak
    ehi: the upper energy bound of the peak
    skimPath: the path to skim file directories
    subsample: the percentage of data to randomly sample from the peak
    '''
    with open(filename, 'wb') as handle:
        for file in tqdm(os.listdir(skimPath)):
            f1 = TFile(os.path.join(skimPath, file))
            tree = f1.Get("skimTree")
            # Change channel[0] == 626 to change detectors.
            array = tree2array(tree,\
                               branches=['run', 'iEvent', 'iHit', 'avse_corr','detName','Final_Energy','tDrift'], #The field to read out from skim file
                               selection='Final_Energy[0] > %.3f && Final_Energy[0] < %.3f && channel[0] == 626 && isGood && isLNFill1==0 && isLNFill2 == 0 && wfDCBits==0 && muVeto==0 && mH == 1 && mL == 1'%(elow, ehi) # The datacleaning and energy cuts
                       )
            if len(array) == 0:
                continue
            run = int(array[0][0])
            gds = GATDataSet(run)
            tt_gat = gds.GetGatifiedChain()
            tt_blt = gds.GetBuiltChain()
            tt_gat.AddFriend(tt_blt)
            tt_gat.GetEntry(0)
            is_ms = tt_blt.run.GetUseMultisampling()
            
            # This part of the code is originated from Clint's waveform reader
            # For each event, read out the waveform accoding to run, iE and iH
            for run, iE, iH, AvsE, detector, energy,tDrift in array:
                #Randomly sample {subsample} fraction of events
#                 if np.random.rand() < (1-subsample):
#                     continue
                #Selecting only the low gain channel
                iH = int(iH[0])
                AvsE = float(AvsE[0])
                energy = float(energy[0])
                detector = str(detector[0])
                tDrift = float(tDrift[0])
                
                tt_gat.GetEntry(iE)
                event = tt_gat.event
                t_off = tt_gat.tOffset.at(iH)
                correctedT0 = tt_gat.correctedT0.at(iH) #Read Corrected T0
                
                if is_ms:
                    wfdown = tt_gat.event.GetWaveform(iH) # downsampled
                    wffull = tt_gat.event.GetAuxWaveform(iH) # fully sampled
                    wf = MJTMSWaveform(wfdown, wffull)
                else:
                    wf = tt_gat.event.GetWaveform(iH)
                period = wf.GetSamplingPeriod()
                wf = wfconverter(wf.GetVectorData())
                
                # int(np.round((correctedT0 + 2140)/10)) convert corrected T0 from time to the index of time samples
                event_dict = {
                    "avse":AvsE,\
                    "run":run,\
                    "event":iE,\
                    "energy":energy,\
                    "detector":detector,\
                    "tDrift":tDrift,\
                    "wf":wf[:-2].astype(np.float32),\
                    "tstart":t_off,\
                    "period":period,\
                    "t0":int(np.round((correctedT0 + 2140)/10))
                }
                pickle.dump(event_dict, handle, protocol=pickle.HIGHEST_PROTOCOL)

In [ ]:
#Extract DEP event on detector P42575A
get_peak("DEP_P42575A_10percent.pickle",1592,1593,skimPath=DS6Cal,subsample=0.5)

In [ ]:
#Extract SEP event on detector P42575A
get_peak("SEP_P42575A_10percent.pickle",1592+511,1593+511,skimPath=DS6Cal,subsample=0.5)

In [ ]:
#Extract Co56 DEP event at 2180keV on detector P42575A
get_peak("DEP_P42575A_Co56.pickle",2180-0.5,2180+0.5,skimPath=DS6Cal56,subsample=1.0)

In [ ]:
#Extract Compton Continuum on detector P42575A
get_peak("CC_Cal.pickle",1989,2089,skimPath=DS6Cal,subsample=1.0)